In [ ]:
#! /usr/bin/python3.5

In [1]:
import math

import pandas as pd

In [2]:
import os
dir_path = os.path.dirname(os.path.realpath(__file__))
path_data_source=dir_path+"/data/database/csv/"
#path_data_source="/home/fsg/Desktop/csv/"
sub_path_data_source_tf="sub_tf/"
sub_path_data_source_idf="sub_idf/"
sub_path_data_source_tfidf="sub_tfidf/"

file_path_tf=path_data_source+sub_path_data_source_tf
file_path_idf=path_data_source+sub_path_data_source_idf

file_names_tf = [os.path.join(file_path_tf, f) 
                      for f in os.listdir(file_path_tf) 
                      if f.endswith(".csv")]
file_names_idf = [os.path.join(file_path_idf, f) 
                      for f in os.listdir(file_path_idf) 
                      if f.endswith(".csv")]

In [3]:
def read_cvs_by_pands(path_database,file_database,index_col, header):
    import csv
    import pandas as pd
    return pd.read_csv(path_database+file_database,index_col=index_col,header=header)

In [4]:
def write_cvs_by_pands(path_database,file_database,header,data_rows):
    import csv
    import pandas as pd
    csv_df=pd.DataFrame(data_rows,columns=header ) 
    csv_df.to_csv(path_database+file_database)

In [54]:
def read_sord_tf_file(path_data_source,sub_path_data_source_tf,i):
    #df=read_cvs_by_pands(path_data_source,sub_path_data_source_tf+"cs"+str(i)+".csv",0,header=0)
    print(path_data_source+sub_path_data_source_tf+i)
    df=read_cvs_by_pands(path_data_source,sub_path_data_source_tf+i,0,header=0)
    #ee.T.sort_index(inplace=True)
    df = df.T.sort_index()
    return df


In [55]:
def read_sord_idf_file(path_data_source,sub_path_data_source_idf,i):
    #df=read_cvs_by_pands(path_data_source,sub_path_data_source_idf+"cs"+str(i)+".csv",0,header=0)
    df=read_cvs_by_pands(path_data_source,sub_path_data_source_idf+i,0,header=0)
    #ee.T.sort_index(inplace=True)
    df = df.sort_index()
    return df

In [56]:
def idf_df(df,D,base):
    #[7/df['0']]
    y = [log_idf(D,x,base) for x in df['0']]
    return y

In [57]:
def log_idf(D,d,base):
    return math.log((D/d), base)

In [58]:
def tf_idf_one(path_data_source,sub_path_data_source_tf,sub_path_data_source_idf,i,sub_path_data_source_tfidf):
    df_tf=read_sord_tf_file(path_data_source,sub_path_data_source_tf,i)
    df_idf=read_sord_idf_file(path_data_source,sub_path_data_source_idf,i)

    if len(df_idf) != 0:
        idf=idf_df(df_idf,len(df_idf),10)

        full_tfidf=[]
        for index in range(len(idf)):
            #print(index)
            tfidf=df_tf[0][index]*idf[index]
            full_tfidf.append(tfidf)


    df_tf_idf=pd.DataFrame(full_tfidf)
    df_tf_idf.index=df_tf.index
    #df_tf_idf.to_csv(path_data_source+sub_path_data_source_tfidf+"cs"+str(i)+".csv")
    df_tf_idf.to_csv(path_data_source+sub_path_data_source_tfidf+i)
    


In [59]:
def name_file(full_name_path):
    d=full_name_path.split("/")
    #print(d)
    name=d[len(d)-1]#.split(".")
    return name

In [60]:
def tf_idfs(list_tf,list_idf):
    for tf_file in list_tf:
        tf_file_name=name_file(tf_file)
        #print(tf_file_name)
        for idf_file in list_idf:
            idf_file_name=name_file(idf_file)
            if tf_file_name == idf_file_name:
                print(idf_file_name)
                tf_idf_one(path_data_source,sub_path_data_source_tf,sub_path_data_source_idf,tf_file_name,sub_path_data_source_tfidf)
            
        
        

In [61]:
tf_idfs(file_names_tf,file_names_idf)

cs6.csv
/home/fsg/Desktop/csv/sub_tf/cs6.csv
cs1.csv
/home/fsg/Desktop/csv/sub_tf/cs1.csv
cs2.csv
/home/fsg/Desktop/csv/sub_tf/cs2.csv
cs4.csv
/home/fsg/Desktop/csv/sub_tf/cs4.csv
cs5.csv
/home/fsg/Desktop/csv/sub_tf/cs5.csv
cs3.csv
/home/fsg/Desktop/csv/sub_tf/cs3.csv
cs7.csv
/home/fsg/Desktop/csv/sub_tf/cs7.csv


In [62]:
import os

import tensorflow as tf
import threading
import timeit
from tensorflow.python.client import timeline

config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True


with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())    
    with tf.device('/gpu'):
        start = timeit.default_timer()
        tf_idfs(file_names_tf,file_names_idf)
        #Your statements here

        stop = timeit.default_timer()

        print ("tttttttttttttt",stop - start )

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


cs6.csv
/home/fsg/Desktop/csv/sub_tf/cs6.csv
cs1.csv
/home/fsg/Desktop/csv/sub_tf/cs1.csv
cs2.csv
/home/fsg/Desktop/csv/sub_tf/cs2.csv
cs4.csv
/home/fsg/Desktop/csv/sub_tf/cs4.csv
cs5.csv
/home/fsg/Desktop/csv/sub_tf/cs5.csv
cs3.csv
/home/fsg/Desktop/csv/sub_tf/cs3.csv
cs7.csv
/home/fsg/Desktop/csv/sub_tf/cs7.csv
tttttttttttttt 0.08581921000040893
